In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression

import pandoc

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

import math

import statsmodels.formula.api as smf

from sklearn.metrics import roc_curve, roc_auc_score
from scipy.stats import chi2
from statsmodels.stats.diagnostic import linear_harvey_collier
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.proportion import proportions_chisquare
from statsmodels.stats.api import het_breuschpagan

import statsmodels.api as sm
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.stats.outliers_influence import summary_table
from statsmodels.stats.diagnostic import linear_reset

from statsmodels.stats.diagnostic import lilliefors
from statsmodels.stats.diagnostic import het_white

from sklearn.metrics import roc_auc_score, roc_curve
from statsmodels.stats.diagnostic import het_breuschpagan

from statsmodels.stats.proportion import proportions_chisquare
from statsmodels.stats.api import het_white

from statsmodels.stats.diagnostic import linear_harvey_collier


from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.api import het_white

from statsmodels.stats.diagnostic import het_white
import os


In [2]:
stats_path = '/Users/caseymoser/Desktop/UFC Analysis/UFC/Go to Ground Analysis/ufc_fight_stats.csv'

results_path = '/Users/caseymoser/Desktop/UFC Analysis/UFC/Go to Ground Analysis/ufc_fight_results.csv'

stats_df = pd.read_csv(stats_path)

results_df = pd.read_csv(results_path)


In [33]:
# Hypothesis is that strikers with more of a grappling base (e.g. bjj blackbelt or wrestling) have a greater finish rate because they are more likely to go to the ground





array(['Tatsuro Taira', 'HyunSung Park', 'Mateusz Rebecki', ...,
       'David Levicki', 'Ray Wizard', 'Sean Daugherty'], dtype=object)

# Note to self: run the scraper and then use that data to add to the fight stats data set
## Note: This code has been commented out to prevent repeat scrapes
## Point of consideration, data does not account for what belt rank they were throughout their career (some fihgters jsut were given belt as they kept winning mma fights so are not necessarily a good grappler)

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from rapidfuzz import fuzz

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from rapidfuzz import fuzz

fighter_names = [str(f).strip() for f in stats_df['FIGHTER'].dropna().unique()]


# Your extract_belt_rank_from_sentences() and get_wrestling_bjj_info() here

def clean_fighter_name(name):
    # Strip nicknames, parentheses etc to improve Wikipedia search
    name = re.sub(r'["\'].*?["\']', '', name)
    name = re.sub(r'\(.*?\)', '', name)
    return name.strip()

def search_wikipedia_page(fighter_name):
    base_url = "https://en.wikipedia.org/w/api.php"
    queries = [
        fighter_name,
        f"{fighter_name} UFC",
        f"{fighter_name} MMA"
    ]
    best_url = None
    best_score = 0

    for query in queries:
        params = {
            'action': 'query',
            'list': 'search',
            'srsearch': query,
            'format': 'json'
        }
        try:
            r = requests.get(base_url, params=params, timeout=10)
            data = r.json()
            pages = data.get('query', {}).get('search', [])
            for page in pages[:5]:
                title = page['title']
                score = fuzz.token_sort_ratio(fighter_name.lower(), title.lower())
                if score > best_score and score > 60:
                    best_score = score
                    best_url = f"https://en.wikipedia.org/wiki/{title.replace(' ', '_')}"
        except Exception as e:
            print(f"Error searching Wikipedia for {fighter_name}: {e}")
            continue
        if best_url:
            break
    return best_url

def scrape_fighters_background(fighter_names, limit=None):
    results = []
    missing = []
    count = 0
    for raw_name in fighter_names:
        name = clean_fighter_name(raw_name)
        print(f"Processing: {raw_name} (cleaned: {name})")
        wiki_url = search_wikipedia_page(name)
        if wiki_url:
            info = get_wrestling_bjj_info(name, wiki_url)
            if info:
                results.append(info)
            else:
                missing.append(raw_name)
        else:
            print(f"No Wikipedia page found for {raw_name}")
            missing.append(raw_name)

        count += 1
        time.sleep(1)  # polite delay

        if limit and count >= limit:
            break

    df = pd.DataFrame(results)
    return df, missing

# Example usage:


df_results, missing_fighters = scrape_fighters_background(fighter_names)

print(f"Scraped {len(df_results)} fighters.")
print(f"Missing or failed for {len(missing_fighters)} fighters: {missing_fighters}")

df_results.to_csv("ufc_fighters_bjj_wrestling_info.csv", index=False)
print("Saved results to ufc_fighters_bjj_wrestling_info.csv")


In [49]:
#note to self: load in new dataset

grappling_path = '/Users/caseymoser/Desktop/UFC Analysis/UFC/Go to Ground Analysis/ufc_fighters_bjj_wrestling_info.csv'

grappling_df = pd.read_csv(grappling_path)



,fighter,bjj_belt_rank,judo_belt_rank,master_of_sport,olympic_wrestler,olympic_judo,wrestling_background,bjj_background,judo_background,other_styles_summary,wiki_url
635,Khamzat Chimaev,brown belt,NaN,No,No,No,Khamzat Chimaev - Wikipedia Jump to content Ma...,Khamzat Chimaev Chimaev in 2022 Born Khamzat K...,[ 30 ] [ 31 ] [ 28 ] Chimaev has competed in a...,[ 30 ] [ 31 ] [ 28 ] Chimaev has competed in a...,https://en.wikipedia.org/wiki/Khamzat_Chimaev


In [65]:
# classifying fighers as strikers, grapplers, or hybrid

# Initial intution is looking at average number of takedowns attempted by a fighter within a fight. Then use that as metric to identify if a fighter is a grappler, striker, or hybrid.
# The above metric can be expanded by factoring in number of striker based attacks like leg attacks, control time etc

stats_df[stats_df['FIGHTER']=='Alex Pereira']

stats_df[stats_df['FIGHTER']=='Khabib Nurmagomedov']
stats_df


stats_df[['TD_Landed', 'TD_Attempted']] = stats_df['TD'].str.split(' of ', expand=True)

# Convert to numeric (handles cases like '---')
stats_df['TD_Landed'] = pd.to_numeric(stats_df['TD_Landed'], errors='coerce')
stats_df['TD_Attempted'] = pd.to_numeric(stats_df['TD_Attempted'], errors='coerce')
stats_df

,EVENT,BOUT,ROUND,FIGHTER,KD,SIG.STR.,SIG.STR. %,TOTAL STR.,TD,TD %,...,REV.,CTRL,HEAD,BODY,LEG,DISTANCE,CLINCH,GROUND,TD_Landed,TD_Attempted
0,UFC Fight Night: Taira vs. Park,Tatsuro Taira vs. HyunSung Park,Round 1,Tatsuro Taira,1.0,6 of 9,66%,12 of 18,2 of 2,100%,...,0.0,3:56,5 of 8,0 of 0,1 of 1,3 of 5,0 of 0,3 of 4,2.0,2.0
1,UFC Fight Night: Taira vs. Park,Tatsuro Taira vs. HyunSung Park,Round 2,Tatsuro Taira,0.0,1 of 2,50%,1 of 2,2 of 3,66%,...,0.0,0:45,1 of 2,0 of 0,0 of 0,1 of 2,0 of 0,0 of 0,2.0,3.0
2,UFC Fight Night: Taira vs. Park,Tatsuro Taira vs. HyunSung Park,Round 1,HyunSung Park,0.0,2 of 5,40%,14 of 17,0 of 0,---,...,0.0,0:00,1 of 3,0 of 0,1 of 2,1 of 3,0 of 1,1 of 1,0.0,0.0
3,UFC Fight Night: Taira vs. Park,Tatsuro Taira vs. HyunSung Park,Round 2,HyunSung Park,0.0,1 of 4,25%,1 of 4,0 of 0,---,...,0.0,0:00,0 of 3,0 of 0,1 of 1,1 of 4,0 of 0,0 of 0,0.0,0.0
4,UFC Fight Night: Taira vs. Park,Mateusz Rebecki vs. Chris Duncan,Round 1,Mateusz Rebecki,0.0,32 of 74,43%,38 of 81,0 of 1,0%,...,0.0,1:00,20 of 61,1 of 2,11 of 11,30 of 72,2 of 2,0 of 0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38785,UFC 2: No Way Out,Johnny Rhodes vs. David Levicki,Round 1,David Levicki,0.0,4 of 5,80%,95 of 102,0 of 0,---,...,0.0,--,4 of 5,0 of 0,0 of 0,1 of 2,2 of 2,1 of 1,0.0,0.0
38786,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,Round 1,Patrick Smith,0.0,1 of 1,100%,1 of 1,0 of 1,0%,...,0.0,--,0 of 0,1 of 1,0 of 0,0 of 0,1 of 1,0 of 0,0.0,1.0
38787,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,Round 1,Ray Wizard,0.0,1 of 1,100%,2 of 2,0 of 0,---,...,0.0,--,0 of 0,0 of 0,1 of 1,1 of 1,0 of 0,0 of 0,0.0,0.0
38788,UFC 2: No Way Out,Scott Morris vs. Sean Daugherty,Round 1,Scott Morris,0.0,1 of 1,100%,2 of 2,1 of 1,100%,...,0.0,--,1 of 1,0 of 0,0 of 0,0 of 0,1 of 1,0 of 0,1.0,1.0
